In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter("ignore")

In [2]:
df = pd.read_excel("cleaned_CB.xlsx")
df

,age,avg_order_value,total_orders,is_fraudulent,loyalty_score,churn_risk,customer_since_month,last_purchase_month,gender_Male,gender_Other,...,country_France,country_Germany,country_India,country_Japan,country_UK,country_USA,preferred_category_Electronics,preferred_category_Fashion,preferred_category_Home,preferred_category_Sports
0,-1.015895,0.150485,8,1,-0.001367,-0.529103,-0.997086,-0.150141,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.270530,-0.029012,10,0,-0.452297,0.347683,-0.770553,-1.009074,0,0,...,0,0,0,0,0,1,1,0,0,0
2,1.389160,-0.158530,6,0,0.518938,-1.468516,-0.883820,0.136169,1,0,...,0,0,0,0,0,0,0,0,0,1
3,-1.015895,0.286960,9,1,1.490173,-0.591730,1.721313,1.567724,0,1,...,0,0,0,1,0,0,1,0,0,0
4,-1.071827,1.871204,16,0,1.004556,-0.842241,-1.563419,1.567724,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0.046803,0.594432,9,0,-1.284785,0.285056,1.551413,1.281413,0,0,...,0,0,0,0,1,0,1,0,0,0
4996,-1.407416,-0.992253,10,0,0.553625,-0.403848,-1.223619,0.995102,0,0,...,0,0,0,0,1,0,0,0,1,0
4997,1.724749,1.534210,16,0,0.206755,-0.341220,0.078947,-1.009074,1,0,...,0,0,0,1,0,0,0,0,0,0
4998,0.773913,1.495854,8,0,-0.105428,-1.218006,-0.260853,0.708791,1,0,...,0,0,0,0,0,1,0,1,0,0


In [3]:
X = df.drop(columns = ['is_fraudulent'])
y = df['is_fraudulent']

**Train_test split**

In [4]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size = 0.8,random_state = 81)

In [5]:
from sklearn.tree import DecisionTreeClassifier,plot_tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

dt_model = DecisionTreeClassifier(random_state=81)
dt_model.fit(X_train,y_train)

ypred_train = dt_model.predict(X_train)
ypred_test = dt_model.predict(X_test)

print('train_accuracy',accuracy_score(ypred_train,y_train))
print('test_accuracy',accuracy_score(ypred_test,y_test))
print('Cross_validation_score',cross_val_score(dt_model,X_train,y_train,cv=5,scoring='accuracy').mean())

train_accuracy 1.0
test_accuracy 0.925
Cross_validation_score 0.9372499999999999


In [6]:
plt.figure(figsize = (30,15),dpi=500)

plot_tree(dt_model,filled = True,
         feature_names = X_train.columns.tolist(),
         class_names = ['Y','N'])
plt.show()

In [7]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
estimator = DecisionTreeClassifier(random_state = 81)
param_grid = {'criterion':['gini','entropy'],
              'max_depth':list(range(1,21))}

grid = GridSearchCV(estimator,param_grid,cv=5,scoring='accuracy')
grid.fit(X_train,y_train)
grid.best_params_

{'criterion': 'gini', 'max_depth': 1}

In [8]:
grid.best_estimator_.feature_importances_

array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.])

In [9]:
feature = pd.DataFrame(data = grid.best_estimator_.feature_importances_,
                       index = X.columns,
                       columns = ['Feature_importance'])

feature_imp = feature[feature['Feature_importance'] > 0]
imp_feature = feature_imp.index.tolist()
imp_feature

['customer_since_month']

In [10]:
X1 = X[imp_feature]

X1_train,X1_test,y1_train,y1_test = train_test_split(X1,y,train_size=0.8,random_state=81)

dt_model = DecisionTreeClassifier(criterion = 'gini',max_depth = 1,random_state = 81)
dt_model.fit(X1_train,y1_train)

y_pred_train = dt_model.predict(X1_train)
y_pred_test = dt_model.predict(X1_test)

print('train_accuracy',accuracy_score(y_pred_train,y1_train))
print('test_accuracy',accuracy_score(y_pred_test,y1_test))
print('cross_validation_score',cross_val_score(dt_model,X1_train,y1_train,cv=5,scoring='accuracy').mean())

train_accuracy 0.975
test_accuracy 0.971
cross_validation_score 0.975
